In [39]:
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.metrics import plot_confusion_matrix
from tensorflow.keras.models import Sequential
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
import scipy
import scipy.integrate 
from IPython.display import Image
from keras import backend as K
from tensorflow import keras
import tensorflow as tf
import numpy as np
import itertools
import glob
import os

In [40]:
class Prediction_Histology:


    def __init__(self, image_gen, shape, model_call, rm):

        self.image_gen = image_gen
        self.shape = shape
        self.model_call = model_call
        self.rm = rm
        

    def preprocess(self):


        def plot_confusion_matrix(cm, classes, normalize=False,
                                  title='Confusion matrix', cmap=plt.cm.Blues):

            plt.imshow(cm, interpolation='nearest', cmap=cmap)
            plt.title(title)
            plt.colorbar()
            tick_marks = np.arange(len(classes))
            plt.xticks(tick_marks, classes, rotation=45)
            plt.yticks(tick_marks, classes)

            if normalize:
                cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
                print("Normalized confusion matrix")
            else:
                print('Confusion matrix, without normalization')

            print(cm)

            thresh = cm.max() / 2.
            for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
                plt.text(j, i, cm[i, j],
                         horizontalalignment="center",
                         color="white" if cm[i, j] > thresh else "black")

            plt.tight_layout()
            plt.ylabel('True label')
            plt.xlabel('Predicted label')
            plt.show()

        os.chdir('/home/lebo/PyHIST')

        train_path = '/home/lebo/PyHIST/training'
        #valid_path = '/home/lebo/PyHIST/validation'
        #test_path = '/home/lebo/PyHIST/testing'

        train_datagen = ImageDataGenerator(preprocessing_function=self.image_gen,
                                           validation_split=0.3) 
        
        train_batch = train_datagen.flow_from_directory(directory=train_path, target_size=self.shape, classes=['stg1', 'stg23', 'stg4'], subset='training', batch_size=64)
        validate_batch = train_datagen.flow_from_directory(directory=train_path, target_size=self.shape, classes=['stg1', 'stg23', 'stg4'], subset='validation', batch_size=64)

        #test_batch = ImageDataGenerator(preprocessing_function=self.image_gen) \
        #.flow_from_directory(directory=test_path, target_size=self.shape,
        #classes=['stg1', 'stg23', 'stg4'], batch_size=15)

    
        imgs, labels = next(train_batch)
        x = self.model_call.layers[self.rm].output

        config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=12,
                                inter_op_parallelism_threads=6,
                                allow_soft_placement=True,
                                device_count = {'CPU': 12})

        session = tf.compat.v1.Session(config=config)

        os.environ["6"] = "6"
        os.environ["10"] = "30"
        os.environ["KMP_SETTINGS"] = "1"
        os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"

        output = Dense(units=3, activation='softmax')(x)
        model = Model(inputs=self.model_call.input, outputs=output)

        model.compile(optimizer=Adam(learning_rate=0.0001),
                            loss='categorical_crossentropy', metrics=['accuracy'])

        history = model.fit(x=train_batch, validation_data=validate_batch, epochs=2, verbose=1)
        predictions = model.predict(x=validate_batch, verbose=0)

        plt.plot(history.history['accuracy'], label='Training')
        plt.plot(history.history['val_accuracy'], label='Validation')

        plt.xlabel('Number of Epochs')
        plt.ylabel('Accuracy(%)')
        plt.tight_layout()

        plt.legend()
        plt.show()

        plt.plot(history.history['loss'], label='Training')
        plt.plot(history.history['val_loss'], label='Validation')

        plt.xlabel('Number of Epochs')
        plt.ylabel('Loss)')
        plt.tight_layout()

        plt.legend()
        plt.show()

        cm = confusion_matrix(y_true=validate_batch.classes, y_pred=np.argmax(predictions, axis=1))
        cm_plot_labels = ['stage1', 'stage23', 'stage4']

        plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

In [41]:
resent_preprocess = tf.keras.applications.resnet_v2.preprocess_input
mobile_preprocess = tf.keras.applications.mobilenet.preprocess_input
incept_preprocess = tf.keras.applications.inception_resnet_v2.preprocess_input

In [42]:
resnet = tf.keras.applications.ResNet152V2()
mobileNet = tf.keras.applications.mobilenet.MobileNet()
incept = tf.keras.applications.InceptionResNetV2()

In [43]:
hist_mobile = Prediction_Histology(mobile_preprocess, (224,224), mobileNet, -2)
hist_resnet = Prediction_Histology(resent_preprocess, (224,224), resnet, -2)
hist_incept = Prediction_Histology(incept_preprocess, (299,299), incept, -2)

In [ ]:
hist_mobile.preprocess()

Found 269402 images belonging to 3 classes.
Found 115456 images belonging to 3 classes.
Epoch 1/2
  88/4210 [..............................] - ETA: 2:45:32 - loss: 1.4363 - accuracy: 0.5836